<a href="https://colab.research.google.com/github/arielwendichansky/DI_Bootcamp/blob/master/week6/day2/daily_challenge/energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pandas
! pip install numpy


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

#Download data and understanding this

In [ ]:
!unzip 'global_power_plant_database_v_1_3.zip'
df = pd.read_csv('global_power_plant_database.csv')
df.head()

Archive:  global_power_plant_database_v_1_3.zip
  inflating: A_Global_Database_of_Power_Plants.pdf  
  inflating: Estimating_Power_Plant_Generation_in_the_Global_Power_Plant_Database.pdf  
  inflating: global_power_plant_database.csv  
  inflating: README.txt              
  inflating: RELEASE_NOTES.txt       


<ipython-input-3-f31ef1201f41>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('global_power_plant_database.csv')


,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.1190,Hydro,NaN,NaN,...,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.670,65.7950,Solar,NaN,NaN,...,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.7920,Solar,NaN,NaN,...,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,NaN,NaN,...,225.06,203.55,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.7170,Hydro,NaN,NaN,...,406.16,357.22,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1


In [ ]:
print(f"There are {len(df)} rows in the file\n")
print("The cells without values are the following\n")
print(df.isnull().sum())
print("\nDescription about each column\n")
print(df.info())


There are 34936 rows in the file

The cells without values are the following

country                               0
country_long                          0
name                                  0
gppd_idnr                             0
capacity_mw                           0
latitude                              0
longitude                             0
primary_fuel                          0
other_fuel1                       32992
other_fuel2                       34660
other_fuel3                       34844
commissioning_year                17489
owner                             14068
source                               15
url                                  18
geolocation_source                  419
wepp_id                           18702
year_of_capacity_data             20049
generation_gwh_2013               28519
generation_gwh_2014               27710
generation_gwh_2015               26733
generation_gwh_2016               25792
generation_gwh_2017               25436
ge

#Preprocessing the data

In [ ]:
cdata = df.copy()
print(df.duplicated().sum())


0


In [ ]:
selected_columns = ['other_fuel1','other_fuel2','other_fuel3','url', 'gppd_idnr','wepp_id','year_of_capacity_data','generation_data_source',
                    'estimated_generation_note_2013','estimated_generation_note_2014','estimated_generation_note_2015',
                    'estimated_generation_note_2016','estimated_generation_note_2017']
cdata.drop(columns = selected_columns,inplace=True )

In [ ]:
cdata['commissioning_year'] = pd.to_datetime(cdata['commissioning_year']).dt.year

In [ ]:
cdata.describe().T


,count,mean,std,min,25%,50%,75%,max
capacity_mw,34936.0,163.355148,489.636072,1.0000,4.900000,16.745000,75.344250,22500.0000
latitude,34936.0,32.816637,22.638603,-77.8470,29.256475,39.727750,46.263125,71.2920
longitude,34936.0,-6.972803,78.405850,-179.9777,-77.641550,-2.127100,49.502675,179.3887
commissioning_year,17447.0,1970.000000,0.000000,1970.0000,1970.000000,1970.000000,1970.000000,1970.0000
generation_gwh_2013,6417.0,592.696107,2174.833482,-947.6000,1.947000,23.426000,199.706000,50834.0000
generation_gwh_2014,7226.0,656.863891,2231.464288,-989.6190,2.260000,23.608500,226.322250,32320.9170
generation_gwh_2015,8203.0,762.368840,2533.459828,-864.4280,2.664500,26.142000,285.862583,37433.6070
generation_gwh_2016,9144.0,693.149763,2404.516759,-768.6200,2.729000,22.462389,249.866750,32377.4770
generation_gwh_2017,9500.0,661.834161,2369.296957,-934.9440,2.467000,17.876500,214.505150,36448.6430
generation_gwh_2018,9637.0,517.320785,1975.302686,-982.6220,2.242000,12.530000,151.117000,35136.0000


In [ ]:
rows_with_missing_values = cdata[cdata.isnull().any(axis=1)]
rows_with_missing_values

Understanding what I can see from those rows where there's no information about the generation of energy for every year. I can see that in most of the cases, no information can be taken from this, therefor I will proceed to drop them.
By doing this only power stations from the US remains

In [ ]:
# columns_na = ['generation_gwh_2013','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016',
#               'generation_gwh_2017','generation_gwh_2018','generation_gwh_2019']
# cdata.dropna(subset=columns_na, inplace=True)
# cdata.isnull().sum()

In [ ]:
energy_gen_columns = ['generation_gwh_2013','generation_gwh_2014','generation_gwh_2015','generation_gwh_2016',
                      'generation_gwh_2017','generation_gwh_2018','generation_gwh_2019','estimated_generation_gwh_2013',
                      'estimated_generation_gwh_2014','estimated_generation_gwh_2015','estimated_generation_gwh_2016', 'estimated_generation_gwh_2017' ]
source_group = cdata.groupby('primary_fuel')[energy_gen_columns].count()
source_group

,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,generation_gwh_2018,generation_gwh_2019,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017
primary_fuel,,,,,,,,,,,,
Biomass,126,136,149,165,156,154,152,0,0,0,0,0
Coal,303,460,602,608,594,481,282,0,0,0,0,2306
Cogeneration,28,29,30,32,32,32,33,0,0,0,0,0
Gas,1661,1742,1924,1920,1933,1851,1775,0,0,0,0,3947
Geothermal,58,59,62,64,63,63,65,0,0,0,0,158
Hydro,1447,1615,1764,1952,1776,1653,1419,7108,7109,7109,7104,7156
Nuclear,58,65,106,108,109,66,58,0,0,0,0,193
Oil,785,784,821,846,832,842,837,0,0,0,0,2263
Other,14,14,17,16,17,15,15,0,0,0,0,42


In [ ]:
# Instantiate LabelEncoder
label_encoder = LabelEncoder()

# Encode 'primary_fuel' column, so we can have a value for each fuel type
cdata['primary_fuel_encoded'] = label_encoder.fit_transform(cdata['primary_fuel'])

# Impute missing values for each column based on 'primary_fuel'
imputer = KNNImputer(n_neighbors=2)

# Impute missing values for each column based on 'primary_fuel_encoded'
for column in energy_gen_columns:

    features = cdata[['primary_fuel_encoded', column]]
    # Impute missing values based on 'primary_fuel_encoded' using KNNImputer
    imputed_values = imputer.fit_transform(features)
    # Replace the imputed values in the original DataFrame
    cdata[column].replace(imputed_values[:, 1], inplace=True)


cdata.drop(columns=['primary_fuel_encoded'], inplace=True)
print(cdata)

In [ ]:
cdata['primary_fuel'].unique()

array(['Hydro', 'Solar', 'Gas', 'Other', 'Oil', 'Wind', 'Nuclear', 'Coal',
       'Waste', 'Biomass', 'Wave and Tidal', 'Petcoke', 'Geothermal',
       'Storage', 'Cogeneration'], dtype=object)

In [ ]:
import pandas as pd

# Assuming `cdata` is your DataFrame and `energy_gen_columns` is a list of columns to be imputed

# Group by 'primary_fuel' and fill missing values with the mean of each group
cdata_filled = cdata.groupby('primary_fuel').apply(lambda group: group.fillna(group.mean()))

# Drop the 'primary_fuel_encoded' column (if it exists)
cdata_filled.drop(columns=['primary_fuel_encoded'], inplace=True, errors='ignore')

# Reset index after groupby operation
cdata_filled.reset_index(drop=True, inplace=True)

print(cdata_filled)